# Distributed Tracing with LastMile Tracing SDK

This Python notebook demonstrates how to implement distributed tracing using the LastMile Tracing SDK. Distributed tracing allows you to track and analyze the flow of requests across multiple services or components in a distributed system.

In this notebook, we showcase a simple example where a client sends a request to a server to generate a riddle. The server uses OpenAI's GPT-3.5-turbo model to generate the riddle and returns it to the client. The LastMile Tracing SDK is used to instrument the code and capture the trace information.

The notebook covers the following key aspects:
- Setting up and configuring the LastMile Tracing SDK
- Instrumenting the server code to create spans and capture trace information
- Instrumenting the client code to propagate the trace context to the server
- Visualizing Trace Data through LastMile's Trace Viewer


Prerequisites:
- Python 3.x
- Required libraries: 
- - LastMile Tracing SDK `lastmile-eval`
- - `multiprocess`

In [ ]:
!pip install lastmile-eval --upgrade
# fork of multiprocessing, uses dill instead of pickle.
!pip install multiprocess
!pip install flask

In [ ]:
# Verify that the LastMile API token is set
from lastmile_eval.rag.debugger.common.utils import get_lastmile_api_token
try:
    get_lastmile_api_token(None)
finally:
    print("LastMile API token is set.")


LastMile API token is set.


In [1]:
import json
import os

import multiprocess
from flask import Flask, request
from openai import OpenAI

from lastmile_eval.rag.debugger.api import LastMileTracer
from lastmile_eval.rag.debugger.tracing import get_lastmile_tracer


def server():
    """
    Starts a Flask server that exposes an endpoint for generating riddles.
    The server uses the LastMileTracer for distributed tracing.
    """
    
    tracer: LastMileTracer = get_lastmile_tracer(
        tracer_name="generate_riddle", # project name
    )
    app = Flask(__name__)

    @app.route("/generate")
    def generate()-> str:
        """
        Endpoint that generates a riddle using OpenAI's GPT-3.5-turbo model.
        If the OPENAI_API_KEY environment variable is not set, a default riddle is returned.
        The generated riddle is returned as a JSON string.
        """
        span_context = request.headers.get("span") # Expect a span to be passed to this endpoint
        with tracer.start_as_current_span("generate_endpoint", context = span_context):
            riddle = ""
            if os.environ.get("OPENAI_API_KEY") is not None:
                response = OpenAI().chat.completions.create(messages = [{"role": "user", "content":"tell me a riddle"}], model = "gpt-3.5-turbo")
                riddle = response.choices[0].message.content or "debugging me can be quite a trick."
            else:
                riddle = "I'm a key that unlocks no doors, But fill me in, and power soars. In the realm where AI plays"
            return json.dumps(riddle)
    app.run(port=1234, debug=False)

# Start Server in a Subprocess to avoid blocking execution of succeeding cells
process = multiprocess.Process(target=server)
process.start()

/Users/ankush/anaconda3/envs/main/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


2024-05-10 12:13:34,114 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-10 12:13:34,202 - https://lastmileai.dev:443 "GET /api/evaluation_projects/list?name=generate_riddle HTTP/1.1" 200 330


 * Serving Flask app '__main__'
 * Debug mode: off


2024-05-10 12:13:34,210 - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:1234
2024-05-10 12:13:34,211 - Press CTRL+C to quit


In [3]:
import requests
from opentelemetry import trace

from lastmile_eval.rag.debugger.api import LastMileTracer
from lastmile_eval.rag.debugger.tracing import export_span, get_lastmile_tracer

tracer2: LastMileTracer = get_lastmile_tracer(
    tracer_name="generate_riddle", # project name
)
@tracer2.start_as_current_span("client")
def client_say_riddle():
    '''
    In order distribute the trace context, we need to export the span context to a string
    and pass it as a header to the server.
    All you need to do is call `export_span(trace.get_current_span())`
    '''
    try:
        print("sending request to subprocess server, with span context.")
        # In order distribute the trace context, we need to export the span context to a string
        # and pass it as a header to the server.
        # All you need to do is call `export_span(trace.get_current_span())``
        response = requests.get('http://127.0.0.1:1234/generate', headers={"span": export_span(trace.get_current_span())})
        return response.text
        
    except Exception as e:
        print(e)
client_say_riddle()

2024-05-10 12:13:22,449 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-10 12:13:22,536 - https://lastmileai.dev:443 "GET /api/evaluation_projects/list?name=generate_riddle HTTP/1.1" 200 330
2024-05-10 12:13:22,550 - Starting new HTTP connection (1): 127.0.0.1:1234


sending request to subprocess server, with span context.


2024-05-10 12:13:24,020 - http://127.0.0.1:1234 "GET /generate HTTP/1.1" 200 166
2024-05-10 12:13:24,022 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-10 12:13:24,101 - https://lastmileai.dev:443 "POST /api/trace/create HTTP/1.1" 200 10
2024-05-10 12:13:24,104 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-10 12:13:24,187 - https://lastmileai.dev:443 "POST /api/rag_query_traces/create HTTP/1.1" 200 495


'"I am taken from a mine, and shut up in a wooden case, from which I am never released, and yet I am used by almost every person. What am I? \\n\\n(answer: pencil lead)"'

In [ ]:
# Check out the UI to see the trace
!rag-debug launch